# Project

...

## Details

...

## Implementation

...

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import itertools
plt.rcParams['figure.figsize'] = (8.0,5.0)
plt.rcParams['font.size'] = 14

In [ ]:
def i_m_test(x_i_j):
    T_i = stats.rankdata(x_i_j, axis=0)-1
    T_v = np.sum(T_i, axis=1)
    
    N = x_i_j.shape[1]
    k = T_i.shape[0]
    
    mu = N*(k-1) / 2
    var = N*(k**2 -1) / 12
    
    T = np.sum((T_v - mu)**2 / var)
    
    return stats.chi2(df=k-1).sf(T)

## Comparison (Kruskal-Wallis)

...

In [ ]:
def kruskal_wallis(x_i_j):
    n_i = np.array([len(xi_j) for xi_j in x_i_j])
    k = len(n_i)
    N = np.sum(n_i)
    
    x_r = np.concatenate(x_i_j)
    R_r = stats.rankdata(x_r)
    i_r = np.concatenate([(i,)*n_i[i] for i in range(k)])
    R_i_j = [R_r[i_r==i] for i in range(k)]
    R_i = np.array([np.sum(Ri_j) for Ri_j in R_i_j])
    
    Rbar = 0.5*(N+1)
    T = (N-1) * np.sum((R_i-n_i*Rbar)**2/n_i) / np.sum((R_r-Rbar)**2)
    
    p = stats.chi2(df=k-1).sf(T)
    
    return p

## Example Comparison

...

In [ ]:
x_i_j = np.array([
    np.array([ 14.97,   5.80,  25.03,   5.50 ]),
    np.array([  5.83,  13.96,  21.96,   55.51 ]),
    np.array([ 17.89,  23.03,  61.09,   18.62 ])
])

In [ ]:
kruskal_wallis(x_i_j)

In [ ]:
i_m_test(x_i_j)

## Experiments

...

In [ ]:
def run_experiment(
    effect_size, x_i_j_k, title,
    n_trials=1000, alpha=0.5
):
    n_trials = 1000  
    alpha = 0.05

    ks_power = []
    im_power = []

    for e in effect_size:
        ks_rejects = 0
        im_rejects = 0

        for _ in range(n_trials):
            x_1 = stats.norm(e, 1).rvs(10)
            x_2 = stats.norm(0, 1).rvs(10)
            x_3 = stats.norm(0, 1).rvs(10)

            x_i_j = np.array([x_1, x_2, x_3])

            if kruskal_wallis(x_i_j) < alpha:
                ks_rejects += 1
            if i_m_test(x_i_j) < alpha:
                im_rejects += 1

        ks_power.append(ks_rejects / n_trials)
        im_power.append(im_rejects / n_trials)

    plt.plot(effect_size, ks_power, label='Kruskal-Wallis')
    plt.plot(effect_size, im_power, label='IM Test')
    plt.xlabel("Effect Size")
    plt.ylabel("Power")
    plt.title(f'{title} | Power Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

### 01 | Normal vs. Normal 

...

#### N=100

In [ ]:
effect_size = np.linspace(0.1, 1, 121)
n=100

x_1 = np.array([stats.norm(e, 1).rvs(n) for e in effect_size])
x_2 = stats.norm(0, 1).rvs(effect_size.shape+(n,))
x_3 = stats.norm(0, 1).rvs(effect_size.shape+(n,))

x_i_j_k = np.array([x_1, x_2, x_3])

run_experiment(effect_size, x_i_j_k, 'Normal vs. Normal')

In [ ]:
run_experiment(effect_size, x_i_j_k, 'Normal vs. Normal')

#### N=10

In [ ]:
effect_size = np.linspace(0.1, 1, 121)
n=10

x_1 = np.array([stats.norm(e, 1).rvs(n) for e in effect_size])
x_2 = stats.norm(0, 1).rvs(effect_size.shape+(n,))
x_3 = stats.norm(0, 1).rvs(effect_size.shape+(n,))

x_i_j_k = np.array([x_1, x_2, x_3])

run_experiment(effect_size, x_i_j_k, 'Normal vs. Normal')

#### N=4

In [ ]:
effect_size = np.linspace(0.1, 1, 121)
n=4

x_1 = np.array([stats.norm(e, 1).rvs(n) for e in effect_size])
x_2 = stats.norm(0, 1).rvs(effect_size.shape+(n,))
x_3 = stats.norm(0, 1).rvs(effect_size.shape+(n,))

x_i_j_k = np.array([x_1, x_2, x_3])

run_experiment(effect_size, x_i_j_k, 'Normal vs. Normal')

#### $\mu_N$=2, N=121

In [ ]:
effect_size = np.linspace(0.1, 2.0, 121)
n=100

x_1 = np.array([stats.norm(e, 1).rvs(n) for e in effect_size])
x_2 = stats.norm(0, 1).rvs(effect_size.shape+(n,))
x_3 = stats.norm(0, 1).rvs(effect_size.shape+(n,))

x_i_j_k = np.array([x_1, x_2, x_3])

run_experiment(effect_size, x_i_j_k, 'Normal vs. Normal')

#### $\mu_N$=2, N=300

In [ ]:
effect_size = np.linspace(0.1, 2.0, 300)
n=100

x_1 = np.array([stats.norm(e, 1).rvs(n) for e in effect_size])
x_2 = stats.norm(0, 1).rvs(effect_size.shape+(n,))
x_3 = stats.norm(0, 1).rvs(effect_size.shape+(n,))

x_i_j_k = np.array([x_1, x_2, x_3])

run_experiment(effect_size, x_i_j_k, 'Normal vs. Normal')

#### $\mu_N$=5, N=1000

In [ ]:
effect_size = np.linspace(0.1, 5.0, 1000)
n=100

x_1 = np.array([stats.norm(e, 1).rvs(n) for e in effect_size])
x_2 = stats.norm(0, 1).rvs(effect_size.shape+(n,))
x_3 = stats.norm(0, 1).rvs(effect_size.shape+(n,))

x_i_j_k = np.array([x_1, x_2, x_3])

run_experiment(effect_size, x_i_j_k, 'Normal vs. Normal')